<a href="https://colab.research.google.com/github/superAI-x/project_a/blob/main/User_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install faiss-cpu

In [20]:
!pip install wikipedia-api

In [24]:
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import wikipediaapi
user_agent = "User_QnA/1.0"
# Load pre-trained BERT model for Question Answering
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Load SBERT model for embedding generation
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to fetch Wikipedia content
def fetch_wikipedia_content(topic):
    wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent)
    page = wiki_wiki.page(topic)
    if page.exists():
        return page.summary  # Return the summary as document content
    return None

# Topics to fetch from Wikipedia
topics = ["Plasmonics","Quantum mechanics", "Quantum computing", "Machine learning", "Graphene", "Multiferroics", "Ferromagnetism"]
documents = []

# Fetch Wikipedia summaries
documents = [fetch_wikipedia_content(topic) for topic in topics if fetch_wikipedia_content(topic)]

# Encode documents into embeddings
doc_embeddings = sbert_model.encode(documents)

dim = doc_embeddings.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(dim)  # L2 distance-based FAISS index
index.add(np.array(doc_embeddings))  # Add document embeddings to FAISS

def retrieve_most_relevant_doc(question, docs, model, index):
    question_embedding = model.encode([question])
    _, best_doc_index = index.search(np.array(question_embedding), 1)  # Retrieve top match
    return docs[best_doc_index[0][0]]

# Sample question
question = "What is Quantum mechanics?"

# Retrieve the most relevant document
relevant_doc = retrieve_most_relevant_doc(question, documents, sbert_model, index)
print("Most Relevant Document:", relevant_doc)

# Get the answer from BERT
output = qa_pipeline(question=question, context=relevant_doc)
print("Answer:", output['answer'])


Device set to use cuda:0


Most Relevant Document: Quantum mechanics is a fundamental theory that describes the behavior of nature at and below the scale of atoms.: 1.1  It is the foundation of all quantum physics, which includes quantum chemistry, quantum field theory, quantum technology, and quantum information science.
Quantum mechanics can describe many systems that classical physics cannot. Classical physics can describe many aspects of nature at an ordinary (macroscopic and (optical) microscopic) scale, but is not sufficient for describing them at very small submicroscopic (atomic and subatomic) scales. Most theories in classical physics can be derived from quantum mechanics as an approximation, valid at large (macroscopic/microscopic) scale.
Quantum systems have bound states that are quantized to discrete values of energy, momentum, angular momentum, and other quantities, in contrast to classical systems where these quantities can be measured continuously. Measurements of quantum systems show characterist